In [1]:
### Imports ###
import pandas as pd
import os
import requests
import json
import time

In [2]:
## Variables ###
api_limit_time_sec = 300
cache_filepath = 'api_cache'
location_record_filepath = os.path.join("/home", "wardrush", "mysite", "data", "GDMBR-2022_rawdata.csv")
spot_column_names = ['@clientUnixTime',
                    'id',
                    'messengerId',
                    'messengerName',
                    'unixTime',
                    'messageType',
                    'latitude',
                    'longitude',
                    'modelId',
                    'showCustomMsg',
                    'dateTime',
                    'messageDetail',
                    'batteryState',
                    'hidden',
                    'messageContent',
                    'altitude']

In [7]:
def get_spot_api_key():
    return '0ArVRndOpKVJJGqc5hN7uGHzV34r5vUko'

def set_last_time_api_called(cache_filepath='api_cache'):
    with open(cache_filepath, 'w') as api_cache:
        api_cache.write(str(int(time.time())))
        
def get_last_time_api_called(cache_filepath='api_cache'):
    with open (cache_filepath) as api_cache:
        return int(api_cache.readlines()[0])

def get_time_since_last_api_call():
    return int(time.time())-get_last_time_api_called()

def get_most_recent_lat_long(location_record_filepath=location_record_filepath):
    # try to grab most recent values
    try:
        df = pd.read_csv(location_record_filepath)
        return (df['latitude'][0], df['longitude'][0])
    # if the file doesn't exist, grab data, create file, and re-call function
    except FileNotFoundError:
        df = create_api_response_dataframe(debug=False)
        save_location_data(location_dataframe=df)
        return get_most_recent_lat_long(location_record_filepath)
    
def get_most_recent_update_time(location_record_filepath=location_record_filepath):
    # try to grab most recent values
    try:
        df = pd.read_csv(location_record_filepath)
        return df['dateTime'][0]
    # if the file doesn't exist, grab data, create file, and re-call function
    except FileNotFoundError:
        df = create_api_response_dataframe(debug=False)
        save_location_data(location_dataframe=df)
        return get_most_recent_update_time(location_record_filepath)

def get_spot_location_data(debug=False, api_limit_time_sec=300):
    # Make the API request to SPOT and return dict from JSON response
    # only make request if at least 5 minutes have passed
    if get_time_since_last_api_call() < api_limit_time_sec:
        print(f'No Request Made--still in API cooldown period. Time left: {api_limit_time_sec-get_time_since_last_api_call()}s')
    else:
        baseurl1 = "https://api.findmespot.com/spot-main-web/consumer/rest-api/2.0/public/feed/"
        baseurl2 = "/message.json"
        url = baseurl1+get_spot_api_key()+baseurl2
        if debug==True: print(url)
        r = requests.get(url)
        set_last_time_api_called()
        return r.json()
        
def create_api_response_dataframe(spot_data=None,
                                  column_names=spot_column_names,
                                  api_limit_time_sec=300,
                                  debug=False):
    if spot_data is None:
        spot_data = get_spot_location_data(api_limit_time_sec=api_limit_time_sec, debug=debug)
    location_df = pd.DataFrame(columns=column_names)
    try:
        for message in spot_data['response']['feedMessageResponse']['messages']['message']:
            #temp
            location_df = location_df.append(message, ignore_index = True)
            #location_df = pd.concat([location_df, pd.DataFrame(message)],axis=0, join='outer')
            location_df = location_df.sort_values(by=['unixTime'], ascending=False)
        if debug==True:
            print(location_df.head())
            return location_df
        else:
            return location_df
    except (ValueError, TypeError) as e:
        print('No data pulled from API at this time. Please try again later')
    except KeyError as e:
        print(f'KeyError: No messages found in feed. It is likely the feed is outdated')
        
        
def save_location_data(api_response_dataframe,
                       location_record_filepath=location_record_filepath):
    # try to grab most recent values from file if it exists
    try:
        read_df = pd.read_csv(location_record_filepath)
        # Concatenate the new data onto the old frame and drop duplicate rows inplace
        combined_df = pd.concat([read_df, api_response_dataframe],axis=0, ignore_index=True).drop_duplicates(subset='unixTime', ignore_index=True, keep='first')
        # Drop old index from combined dataframe
        combined_df = combined_df.sort_values(by=['unixTime'], ascending=False).reset_index(drop=True)
        # Do not add new index when saving CSV
        combined_df.to_csv(location_record_filepath, index=False)
    # if the file doesn't exist, create file and save
    except FileNotFoundError:
        # Do not add new index when saving CSV
        api_response_dataframe = create_api_response_dataframe()
        api_response_dataframe.sort_values(by=['unixTime'], ascending=False).reset_index(drop=True).to_csv(location_record_filepath, index=False)

In [8]:
df = create_api_response_dataframe(api_limit_time_sec=30, debug=False)
save_location_data(api_response_dataframe=df)
display(df)

KeyError: No messages found in feed. It is likely the feed is outdated


None

In [19]:

read_df = pd.read_csv(location_record_filepath).drop_duplicates(subset='unixTime', ignore_index=True, keep='first')
display(read_df)

,@clientUnixTime,id,messengerId,messengerName,unixTime,messageType,latitude,longitude,modelId,showCustomMsg,dateTime,messageDetail,batteryState,hidden,messageContent,altitude
0,0,1767536410,0-3777079,WardSpotX,1652469935,TRACK,32.81345,-96.77319,SPOTXB,N,2022-05-13T19:25:35+0000,NaN,GOOD,0,NaN,131
1,0,1767536424,0-3777079,WardSpotX,1652469335,TRACK,32.81324,-96.77319,SPOTXB,N,2022-05-13T19:15:35+0000,NaN,GOOD,0,NaN,0
2,0,1767526930,0-3777079,WardSpotX,1652468732,TRACK,32.81327,-96.77307,SPOTXB,N,2022-05-13T19:05:32+0000,NaN,GOOD,0,NaN,173
3,0,1767401910,0-3777079,WardSpotX,1652453684,TRACK,32.90903,-96.76312,SPOTXB,N,2022-05-13T14:54:44+0000,NaN,GOOD,0,NaN,158
4,0,1767387805,0-3777079,WardSpotX,1652451969,TRACK,32.90894,-96.76175,SPOTXB,N,2022-05-13T14:26:09+0000,NaN,GOOD,0,NaN,197
5,0,1767382977,0-3777079,WardSpotX,1652451380,TRACK,32.84999,-96.77118,SPOTXB,N,2022-05-13T14:16:20+0000,NaN,GOOD,0,NaN,191
6,0,1767376464,0-3777079,WardSpotX,1652450557,TRACK,32.84443,-96.77361,SPOTXB,N,2022-05-13T14:02:37+0000,NaN,GOOD,0,NaN,167
7,0,1767346851,0-3777079,WardSpotX,1652446705,TRACK,32.84473,-96.77347,SPOTXB,N,2022-05-13T12:58:25+0000,NaN,GOOD,0,NaN,38
8,0,1764256934,0-3777079,WardSpotX,1651938540,TRACK,32.81092,-96.78180,SPOTXB,N,2022-05-07T15:49:00+0000,NaN,GOOD,0,NaN,158
9,0,1763984598,0-3777079,WardSpotX,1651881129,TRACK,32.79184,-96.79358,SPOTXB,N,2022-05-06T23:52:09+0000,NaN,GOOD,0,NaN,143


In [147]:
print(get_most_recent_lat_long())

(32.81092, -96.7818)


<ipython-input-146-82526aab884b>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  location_df = location_df.append(message, ignore_index = True)
<ipython-input-146-82526aab884b>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  location_df = location_df.append(message, ignore_index = True)
<ipython-input-146-82526aab884b>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  location_df = location_df.append(message, ignore_index = True)
<ipython-input-146-82526aab884b>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  location_df = location_df.append(message, ignore_index = True)
<ipython-input-146-82526aab884b>:47: FutureWarning: The fram

In [99]:
# importing the module
import pandas as pd
  
# creating the Series
series1 = df.iloc[[2]]
display('series1:', series1)
series2 = df.iloc[[4]]
display('series2:', series2)
  
# concatenating
df2 = pd.concat([series1, series2],axis=0, join='outer').sort_values(by=['unixTime'], ascending=False)
df2 = pd.concat([df2, df.iloc[[7]]],axis=0, join='outer').sort_values(by=['unixTime'], ascending=False)
display('After concatenating:')
display(df3)

'series1:'

,@clientUnixTime,id,messengerId,messengerName,unixTime,messageType,latitude,longitude,modelId,showCustomMsg,dateTime,messageDetail,batteryState,hidden,messageContent,altitude
7,0,1763564577,0-3777079,WardSpotX,1651844701,TRACK,32.8133,-96.77307,SPOTXB,N,2022-05-06T13:45:01+0000,,GOOD,0,NaN,179


'series2:'

,@clientUnixTime,id,messengerId,messengerName,unixTime,messageType,latitude,longitude,modelId,showCustomMsg,dateTime,messageDetail,batteryState,hidden,messageContent,altitude
5,0,1763706973,0-3777079,WardSpotX,1651860084,TRACK,32.81368,-96.77023,SPOTXB,N,2022-05-06T18:01:24+0000,,GOOD,0,NaN,0


'After concatenating:'

,@clientUnixTime,id,messengerId,messengerName,unixTime,messageType,latitude,longitude,modelId,showCustomMsg,dateTime,messageDetail,batteryState,hidden,messageContent,altitude
2,0,1763761932,0-3777079,WardSpotX,1651861910,TRACK,32.78876,-96.79974,SPOTXB,N,2022-05-06T18:31:50+0000,,GOOD,0,NaN,389
5,0,1763706973,0-3777079,WardSpotX,1651860084,TRACK,32.81368,-96.77023,SPOTXB,N,2022-05-06T18:01:24+0000,,GOOD,0,NaN,0
7,0,1763564577,0-3777079,WardSpotX,1651844701,TRACK,32.8133,-96.77307,SPOTXB,N,2022-05-06T13:45:01+0000,,GOOD,0,NaN,179


In [142]:
pd.read_csv('test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'test.csv'

In [11]:
from datetime import datetime
import pandas as pd
import os

location_record_filepath = os.path.join("/home", "wardrush", "mysite", "data", "GDMBR-2022_rawdata.csv")
df = pd.read_csv(location_record_filepath)
timevalue = df['unixTime'][0]

In [12]:
print(datetime.utcfromtimestamp(timevalue).strftime('%m/%m/%d %H:%M:%S'))

2022-05-13 19:25:35
